In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10154,2024-05-03,Brasil Lbf Feminina,21:00,Campinas F,Corinthians F,1.55,2.34,138.5,1.86,1.84,2.5,1.84,2.08,dW1jBdvS,0.645161,0.427350,0.537634,0.543478,0.072512,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.287205,0.007644,0.086585,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10155,2024-05-03,Eua Nba,20:00,Orlando Magic,Cleveland Cavaliers,1.59,2.45,199.5,1.80,1.95,-4.5,1.95,3.30,6sXxmCP8,0.628931,0.408163,0.555556,0.512821,0.037094,0.4,0.4,NaN,NaN,210.372,38.528098,0.183143,208.186,61.043275,0.293215,182.102,216.016,196.00,192.24,0.0,0.0,0.0,0.0,0.301045,0.056569,0.363655,-1.47,-0.294,-2.006803,0.615040,0.6,-0.015040,-5.00,-1.000,-1.450000,0.604312,0.3,-0.304312
10156,2024-05-03,Eua Nba,22:30,Dallas Mavericks,Los Angeles Clippers,1.30,3.72,205.5,1.80,1.95,-7.5,1.84,7.50,bkP5irdd,0.769231,0.268817,0.555556,0.512821,0.038048,0.0,0.0,NaN,NaN,172.788,20.317162,0.117584,231.114,27.217783,0.117768,186.612,214.932,154.53,234.90,0.0,0.0,0.0,0.0,0.681752,0.056569,0.857007,3.45,0.690,0.434783,0.641274,0.9,0.258726,0.98,0.196,13.877551,0.579714,0.7,0.120286
10157,2024-05-03,Bolívia Libobasquet,20:00,Carl A-Z,Kinwa,1.92,1.80,163.5,1.85,1.85,3.5,1.55,1.47,bmvQQEs6,0.520833,0.555556,0.540541,0.540541,0.076389,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.045620,0.000000,0.037463,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10158,2024-05-03,Chile Lnb,21:30,Ancud,U. De Concepcion,1.24,3.77,169.5,1.83,1.87,3.5,1.83,2.35,xxnsGwBN,0.806452,0.265252,0.546448,0.534759,0.071704,0.0,0.0,NaN,NaN,126.712,36.741422,0.289960,126.294,35.510449,0.281173,0.000,0.000,87.72,128.79,0.0,0.0,0.0,0.0,0.714164,0.015289,0.175931,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10159,2024-05-03,Espanha Leb Ouro,16:00,Real Valladolid,Oviedo,1.65,2.15,154.5,1.85,1.85,-2.5,1.85,2.43,YktxNMsf,0.606061,0.465116,0.540541,0.540541,0.071177,0.8,1.0,NaN,NaN,165.478,61.970308,0.374493,162.352,8.573306,0.052807,160.342,151.776,204.40,165.17,0.0,0.0,0.0,0.0,0.186081,0.000000,0.191646,0.34,0.068,9.558824,0.636714,0.7,0.063286,-5.00,-1.000,-1.150000,0.000000,0.0,0.000000
10160,2024-05-03,Filipinas Copa Das Filipinas,08:30,NLEX Road Warriors,Rain Or Shine Paint.,2.34,1.57,212.5,1.88,1.86,2.5,2.02,1.39,6ej8PTgd,0.427350,0.636943,0.531915,0.537634,0.064293,0.0,0.0,NaN,NaN,NaN,NaN,NaN,202.706,43.060505,0.212428,0.000,0.000,0.00,173.80,0.0,0.0,0.0,0.0,0.278502,0.007563,0.261277,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10161,2024-05-03,Finlândia Korisliiga,12:30,Kataja,BC Nokia,1.95,1.80,165.5,1.80,1.86,1.5,1.80,1.64,fkCKlh9F,0.512821,0.555556,0.555556,0.537634,0.068376,0.8,0.6,NaN,NaN,202.556,95.268414,0.470331,161.790,19.160719,0.118430,198.590,147.900,141.60,185.07,0.0,0.0,0.0,0.0,0.056569,0.023184,0.065777,2.34,0.468,2.029915,0.564864,0.6,0.035136,-0.18,-0.036,-22.222222,0.597228,0.7,0.102772
10162,2024-05-03,França Pro B,15:00,Aix-Maurienne,Fos-sur-Mer,2.02,1.72,162.5,1.80,1.86,1.5,1.87,1.59,r1e30g0D,0.495050,0.581395,0.555556,0.537634,0.076445,0.6,0.2,NaN,NaN,247.616,56.429957,0.227893,164.026,45.088082,0.274884,184.994,189.508,169.29,153.30,0.0,0.0,0.0,0.0,0.113440,0.023184,0.114445,1.49,0.298,3.422819,0.000000,0.0,0.000000,0.16,0.032,22.500000,0.000000,0.0,0.000000
10163,2024-05-03,Islândia Premier League,16:15,Njardvik,Va

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Brasil Lbf Feminina,Campinas F,Corinthians F,138.5,1.86,0.9949,Over
1,20:00,Eua Nba,Orlando Magic,Cleveland Cavaliers,199.5,1.80,1.0000,Over
2,22:30,Eua Nba,Dallas Mavericks,Los Angeles Clippers,205.5,1.80,1.0000,Over
3,21:30,Chile Lnb,Ancud,U. De Concepcion,169.5,1.83,1.0000,Over
4,16:00,Espanha Leb Ouro,Real Valladolid,Oviedo,154.5,1.85,1.0000,Over
5,16:15,Islândia Premier League,Njardvik,Valur,183.5,1.85,1.0000,Over
6,07:15,Israel Superliga,H. Afula,Nes Ziona,163.5,1.80,1.0000,Over
7,15:45,Reino Unido Bbl,Bristol Flyers,Cheshire Phoenix,169.5,1.80,0.9970,Over
8,13:00,Romênia Divisão A,Voluntari,CSM Oradea,158.5,1.87,1.0000,Over
